載入模組

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.preprocessing  import LabelEncoder #01轉換
from sklearn import preprocessing #標準化
from sklearn.model_selection import train_test_split #切分 

import statsmodels.api as sm 
# from sklearn.linear_model import LinearRegression #sk線性回歸
# from yellowbrick.regressor import ResidualsPlot #殘插圖

# from sklearn.metrics import mean_squared_error #mse
# from statsmodels.stats.outliers_influence import variance_inflation_factor #VIF

載入資料\資料前處理

In [15]:
data = pd.read_csv('CrabAgePrediction.csv') #原資料
df = data.drop(data[data['Sex']=='I'].index) #去I資料
labelencoder = LabelEncoder()
df['Sex'] = labelencoder.fit_transform(df['Sex']) #01轉換
# df.head(5)
# df.describe().style.format('{:2.4f}')
X = df.iloc[:,0:6]
Y = df['Age']

切分資料 9:1

In [16]:
df_train, df_test = train_test_split(df, random_state = True, train_size=0.9)
print(len(df_train))

2394


刪除3*IQR離群值

In [17]:
j = []
IQR = df_train.quantile(0.75)-df_train.quantile(0.25)
extreme_outlier = (df_train[(df_train > df_train.quantile(0.75)+3*IQR)|(df_train < df_train.quantile(0.25)-3*IQR)])

for i in extreme_outlier.iloc[:,1:9]:
    j.extend((extreme_outlier[i].dropna()).index)
del_list = set(j)
df_train = df_train.drop(del_list)
print(len(df_train))
df_train.describe().style.format('{:2.4f}')

2364


,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
count,2364.0000,2364.0000,2364.0000,2364.0000,2364.0000,2364.0000,2364.0000,2364.0000,2364.0000
mean,0.5385,1.4242,1.1165,0.3846,28.7177,12.3911,6.2967,8.2285,10.8050
std,0.4986,0.2357,0.1941,0.0776,12.6199,5.8999,2.8449,3.5690,2.8534
min,0.0000,0.4375,0.3375,0.1000,0.8647,0.2835,0.1559,0.2835,3.0000
25%,0.0000,1.2875,1.0000,0.3375,19.7667,8.0761,4.2949,5.7266,9.0000
50%,1.0000,1.4625,1.1500,0.3875,28.4345,12.2115,6.1660,8.0796,10.0000
75%,1.0000,1.5875,1.2500,0.4375,36.4823,16.0954,8.0973,10.3476,12.0000
max,1.0000,2.0375,1.6250,0.6250,72.4330,38.3002,18.1862,23.1048,21.0000


對X標準化

In [18]:
zscore = preprocessing.StandardScaler()
df_train.iloc[:,1:8] = zscore.fit_transform(df_train.iloc[:,1:8])
df_train.head(5)
df_train.describe().style.format('{:2.4f}')

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
count,2364.0000,2364.0000,2364.0000,2364.0000,2364.0000,2364.0000,2364.0000,2364.0000,2364.0000
mean,0.5385,-0.0000,-0.0000,-0.0000,0.0000,0.0000,0.0000,-0.0000,10.8050
std,0.4986,1.0002,1.0002,1.0002,1.0002,1.0002,1.0002,1.0002,2.8534
min,0.0000,-4.1868,-4.0135,-3.6674,-2.2075,-2.0526,-2.1590,-2.2266,3.0000
25%,0.0000,-0.5801,-0.6000,-0.6069,-0.7094,-0.7315,-0.7038,-0.7012,9.0000
50%,1.0000,0.1624,0.1728,0.0374,-0.0224,-0.0304,-0.0460,-0.0417,10.0000
75%,1.0000,0.6928,0.6880,0.6817,0.6154,0.6280,0.6331,0.5939,12.0000
max,1.0000,2.6022,2.6202,3.0979,3.4647,4.3923,4.1802,4.1691,21.0000


In [19]:
df_train.to_csv('訓練集.csv')
df_test.to_csv('測試集.csv')

## 後處理

逐步回歸

In [21]:
X_add = pd.DataFrame()
X_del = pd.DataFrame()
TP = pd.DataFrame(columns=['name','tvalue','pvalue'])
TP_del = pd.DataFrame(columns=['name','tvalue','pvalue'])

In [22]:
Step = 0
b = True
while b:
    Step += 1
    print('\nStep:',Step)
    a = True
    TP.drop(TP.index,inplace=True)
    TP_del.drop(TP_del.index,inplace=True)
    # 加入項
    for i in X.columns:
        if X_add.empty == True:
            x = X[i]
        else:
            x = X_add.join(X[i])
        x = sm.add_constant(x)
        y = Y
        model = sm.OLS(y, x).fit()
        # print(model.summary())
        TP_new = pd.DataFrame([[i, abs(model.tvalues[i]), abs(model.pvalues[i])]]
                                ,columns=['name','tvalue','pvalue'])
        TP = pd.concat([TP,TP_new], ignore_index=True)
    
    while a:
        if TP.size == 0:
            b = False
            a = False
        else:
            tp = TP[TP['tvalue'] == max(TP['tvalue'].values)]
            if (tp['pvalue'].values < 0.1):
                X_add[tp['name']] = X[tp['name']]
                X = X.drop(columns = (tp['name']))
                TP = TP.drop(TP[TP['name'].values == tp['name'].values].index)
                a = False
            else:
                TP = TP.drop(TP[TP['name'].values == tp['name'].values].index)
        print('剩餘項個數',TP.size/3)
    print('剩餘項\n',TP)       

    # 刪除項 
    for j in X_add.columns:
        x = X_add
        x = sm.add_constant(x)
        y = Y
        model = sm.OLS(y, x).fit()
        # print(model.summary())
        TP_del_new = pd.DataFrame([[j,abs(model.tvalues[j]), abs(model.pvalues[j])]]
                                    ,columns=['name','tvalue','pvalue']
                                    ,index = [j])
        TP_del = pd.concat([TP_del,TP_del_new])

        if (TP_del.loc[j,'pvalue'] > 0.15):
            print('刪除\n',TP_del.loc[j])
            X[j] = X_add[j]
            X_add = X_add.drop(columns = j)
            TP_del = TP_del.drop(j)
# print(model.summary())
    print('\n保留項\n',TP_del,'\n-----------------------')


Step: 1
剩餘項個數 5.0
剩餘項
              name     tvalue    pvalue
0             Sex   3.505651  0.000463
1          Length  16.637617       0.0
3          Height  17.765449       0.0
4          Weight  17.671202       0.0
5  Shucked Weight   8.597004       0.0

保留項
               name     tvalue pvalue
Diameter  Diameter  18.387762    0.0 
-----------------------

Step: 2
剩餘項個數 4.0
剩餘項
      name    tvalue    pvalue
0     Sex  1.940476  0.052428
1  Length  5.589309       0.0
2  Height  6.966383       0.0
3  Weight  2.207932  0.027334

保留項
                           name     tvalue pvalue
Diameter              Diameter  22.588671    0.0
Shucked Weight  Shucked Weight  15.197221    0.0 
-----------------------

Step: 3
剩餘項個數 3.0
剩餘項
      name    tvalue    pvalue
0     Sex  0.371175  0.710537
1  Length   0.98915   0.32268
2  Height  9.229918       0.0

保留項
                           name     tvalue pvalue
Diameter              Diameter   5.491791    0.0
Shucked Weight  Shucked Weight  31.73

殘差

In [25]:
# visualizer = ResidualsPlot(LinearRegression(), hist = False, qqplot = True)
# visualizer.fit(X_add,Y)
# visualizer.poof()

VIF

In [29]:
# vif = pd.DataFrame()
# vif['variable'] = df.iloc[:,1:9].columns
# vif["VIF"] = [variance_inflation_factor(df.iloc[:,1:9], i) for i in range(len(df.iloc[:,1:9].columns))]
# vif